## Machine Learning Record Mining

Project to create a pipeline that uses GeoDeepDive's output to find Unaquired Sites for Neotoma.

Using NLP parsed text and a Data Science approach, identify whether a paper is suitable for Neotoma and detect features such as 'Site Name', 'Location', 'Age Span' and 'Site Descriptions'.

In [2]:
# Loading libraries

import numpy as np
import pandas as pd
import csv
import psycopg2
import re

# Load Postgres Server
from src.config import config

In [24]:
# Options for DF display
pd.set_option('display.max_colwidth', 300)
pd.set_option('display.max_rows', 10)

#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)

## Loading and viewing the Data

### Loading NLP Sentences

In [4]:
# Connect to PostgreSQL server from terminal:
# pg_ctl -D PSQL_Data -l logfile start

try:
    params = config()
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(**params)
    # Create a new cursor
    cur = conn.cursor()
    nlp_sentences = pd.read_sql_query('''SELECT * FROM sentences;''', conn)
    # Close the cursor and connection to so the server can allocate
    # bandwidth to other requests
    cur.close()
    conn.close()
    print('Data obtained from PostgreSQL')
    
# If no SQL db, load from a file
except:
    header_list = ["_gddid", "sentid", "wordidx", "words", "part_of_speech", "special_class", 
               "lemmas", "word_type", "word_modified"]
    nlp_sentences = pd.read_csv("../Do_not_commit_data/sentences_nlp352", sep='\t', names = header_list)
    nlp_sentences = nlp_sentences.replace('"', '', regex = True)\
                                 .replace('\{', '', regex = True)\
                                 .replace('}', '', regex = True)\
                                 .replace(',', ',', regex = True)
    nlp_sentences['wordidx']= nlp_sentences['wordidx'].str.split(",")
    nlp_sentences['words']= nlp_sentences['words'].str.split(",")
    nlp_sentences['poses']= nlp_sentences['poses'].str.split(",")
    nlp_sentences['ners']= nlp_sentences['ners'].str.split(",")
    nlp_sentences['lemmas']= nlp_sentences['lemmas'].str.split(",")
    nlp_sentences['dep_paths']= nlp_sentences['dep_paths'].str.split(",")
    nlp_sentences['dep_parents']= nlp_sentences['dep_parents'].str.split(",")
    print('Data obtained from text file')

Data obtained from PostgreSQL


In [18]:
nlp_sentences.head(3)

,_gddid,sentid,wordidx,words,part_of_speech,special_class,lemmas,word_type,word_modified
0,54b43266e138239d8684efed,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[Available, online, at, ww...","[JJ, NN, IN, NNP, NNP, NNP...","[O, O, O, O, O, O, NUMBER,...","[available, online, at, ww...","[dep, dep, dep, dep, dep, ...","[218, 218, 218, 218, 218, ..."
1,54b43266e138239d8684efed,2,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[The, Chihuahueños, Bog, r...","[DT, NNP, NN, NN, VBZ, TO,...","[O, O, O, O, O, O, O, NUMB...","[the, Chihuahueños, bog, r...","[det, compound, compound, ...","[4, 4, 4, 5, 0, 11, 11, 11..."
2,54b43266e138239d8684efed,3,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[An, Artemisia, steppe, ,,...","[DT, NNP, NN, ,, RB, DT, J...","[O, O, O, O, O, O, O, LOCA...","[a, Artemisia, steppe, ,, ...","[det, compound, nsubj, , a...","[3, 3, 10, 0, 9, 9, 9, 9, ..."


In [19]:
# Running this takes forever. Do not run unless necessary
# nlp_sentences.describe(include = 'all')

In [17]:
avg_sentences = nlp_sentences.groupby('_gddid')['_gddid','sentid'].transform('max')
avg_sentences.drop_duplicates(subset='_gddid', keep='first', inplace=True)
avg_sentences.describe()

,sentid
count,218.000000
mean,513.600917
std,232.460891
min,2.000000
25%,397.250000
50%,504.000000
75%,638.250000
max,1330.000000


From the Summary Statistics, despite having 111,965 sentences, there are only 218 unique articles (`_gddid`). The article with most sentences, had 1330 sentences; that's the max number found in `sentid`.  
In order to know the average of sentences, I grouped by `_gddid` and then, obtained the number of sentences per article. After that, I just got the mean through a Summary of statistics to know that on average, each article has 514 sentences.

### Loading Bibliography Data

In [20]:
import json
with open('../Do_not_commit_data/bibjson', 'r') as f:
    bib_dict = json.load(f)

In [60]:
# Normalizing data so that we have access to the 'identifier'

# TODO Load into SQL server and connect through SQL
bibliography = pd.io.json.json_normalize(bib_dict,'identifier',['publisher', 'title', ['journal','name', 'name'], ['author'], 'year', 'number', 'volumne', ['link'], '_gddid', 'type', 'pages'], record_prefix='_', errors='ignore')

bibliography['link'] = bibliography['link'].astype(str)

url = bibliography['link'].str.split(", ", expand = True)

bibliography['link_url'] = url[0]
bibliography['link_type'] = url[1]

bibliography['link_url'] = bibliography['link_url'].replace("\[{'url': '", "", regex = True)\
                                                   .replace("'", "", regex = True)

bibliography['link_type'] = bibliography['link_type'].replace("'type': '", "", regex = True)\
                                                     .replace("'}]", "", regex = True)

bibliography['author'] = bibliography['author'].astype(str)
bibliography['author'] = bibliography['author'].replace("\[{'name': '", "", regex = True)\
                                               .replace("{'name': '", "", regex = True)\
                                               .replace("'},", ";", regex = True)\
                                               .replace("'}]", "", regex = True)
bibliography = bibliography[['_type', '_id', 'publisher', 'title', 'journal.name.name',	'author', 'year', 'number', 'volumne', '_gddid', 'type', 'pages', 'link_url', 'link_type']]

bibliography.head(3)

,_type,_id,publisher,title,journal.name.name,author,year,number,volumne,_gddid,type,pages,link_url,link_type
0,doi,10.1016/S0277-3791(99)00007-4,Elsevier,"Palaeoclimate, chronology and vegetation history of the Weichselian Lateglacial: comparative analysis of data from three cores at Lago Grande di Monticchio, southern Italy",Quaternary Science Reviews,"Huntley, B.; Watts, W.A.; Allen, J.R.M.; Zolitschka, B.",1999,7,NaN,550453fde1382326932d85f7,article,945--960,http://www.sciencedirect.com/science/article/pii/S0277379199000074,publisher
1,doi,10.1139/b92-002,Canadian Science Publishing,"Holocene history of forest trees on the Bruce Peninsula, southern Ontario",Canadian Journal of Botany,"Bennett, K. D.",1992,1,NaN,578b5aabcf58f1587003efba,article,6--18,http://www.nrcresearchpress.com/doi/abs/10.1139/b92-002,publisher
2,doi,10.1016/0031-0182(80)90043-7,Elsevier,Glacial sequence and environmental history in the Sierra Nevada del cocuy (Colombia),"Palaeogeography, Palaeoclimatology, Palaeoecology","Van Der Hammen, T.; Barelds, J.; De Jong, H.; De Veer, A.A.",1980,NaN,NaN,54b43244e138239d868493cd,article,247--340,http://www.sciencedirect.com/science/article/pii/0031018280900437,publisher


In [63]:
bibliography.describe(include = 'all')

,_type,_id,publisher,title,journal.name.name,author,year,number,volumne,_gddid,type,pages,link_url,link_type
count,221,221,221,221,221,221,221,183,0,221,221,216,221,220
unique,1,221,6,221,41,213,47,28,0,221,1,214,221,1
top,doi,10.1002/jqs.910,Elsevier,"A 30,000-year pollen record from Mire Kupena, Western Rhodopes Mountains (south Bulgaria)",Quaternary Research,"Markgraf, Vera",2011,3,NaN,56f907c7cf58f176c533d523,article,1--13,http://www.sciencedirect.com/science/article/pii/027737919390043L,publisher
freq,221,1,138,1,44,3,14,40,NaN,1,221,2,1,220


## EDA

Reviewing our data includes skimming through some papers online and seeing if the data is consistent with our NLP Sentences dataframe.

From there, we can also visualize what we would like our model to predict: 'Location', 'Site Name', 'Age Span', and 'Site Description' from a "Human perspective".

In [8]:
# TODO Add number of articles in file and some summary Stats

### Finding Latitude and Longitud via REGEX 

After reviewing some PDF's and viewing the data parsing, I noticed some possible patterns for coordinates.

<img src="figures/img/00_snippet_of_pdf.png" alt="Drawing" style="width: 300px;"/>

Which translates to our pandas dataframe as:

![alt text](figures/img/01_snippet_in_sql.png "PandasVersion")

As a first problem, we can notice that the degrees symbol `°` was translated as a `3`. On some other occassions, it is confused as an `o` or a middle point `◦`. For `'`, this symbol got confused by the word `cents`.

I am still going to use normal REGEX to find coordinates in the most intuitive way. 

A coordinate for longitude is a number ranging from -180° to 180°. It has three components, `°, ',"` and it covers the position in E or W.

Latitude goes from -90° to 90°, has the same three components and goes from S to N.

Using REGEX, and assuming data consistency, the following code should be able to extract coordinates. 

In [9]:
dms_regex = r"([-]?\d{1,3}[°|′|\'|,]{0,3}\d{1,2}[,|′|\']{1,3}\d{0,2}[,|′|\']{1,4}[NESWnesw]?[\s|,|\']+?[-]?\d{1,3}[°|,|′|\']+\d{1,2}[,|′|\']+\d{0,2}[,|′|\'][,|′|\']{0,4}[NESWnesw]?)"

dd_regex = "[\\{,][-]?[1]?[0-9]{1,2}\\.[0-9]{1,}[,]?[NESWnesw],"

In [10]:
# TODO Find REGEX of couples of numbers (123, 234)
nlp_sentences['liststring'] = nlp_sentences['words'].apply(lambda x: ','.join(map(str, x)))
nlp_sentences['dms_regex'] = nlp_sentences['liststring'].str.findall(dms_regex)


#nlp_sentences[['dms_regex', 'words']]

nlp_sentences[['liststring', 'words', 'dms_regex']].head(3)

,liststring,words,dms_regex
0,"Available,online,at,www.sc...","[Available, online, at, ww...",[]
1,"The,Chihuahueños,Bog,recor...","[The, Chihuahueños, Bog, r...",[]
2,"An,Artemisia,steppe,,,then...","[An, Artemisia, steppe, ,,...",[]


In [11]:
italy_example = nlp_sentences[nlp_sentences['_gddid'] == '550453fde1382326932d85f7']
italy_example = italy_example[italy_example['sentid'] == 10]
italy_example[['words', 'liststring', 'dms_regex']]

,words,liststring,dms_regex
94875,"[Introduction, Laghi, di, ...","Introduction,Laghi,di,Mont...","[403,56,40,N,,,153,36,48,E]"


In [12]:
italy_example['words'].astype(str).str.len()

94875    391
Name: words, dtype: int64

### Problems with the REGEX approach

As seen before, extracting information using REGEX is very complicated. We would require a lot of queries that would retrieve each particular case.

A possible alternative could be using NER, Named Entity Recognition.

In [13]:
def order_article(article_id):
    '''
    Function to find an article by its gddid in the NLP sentences and have it displayed in order
    
    Keyword arguments:
    article_id -- gddid
    
    Returns:
    article ordered by sentence index
    '''
    article = nlp_sentences[nlp_sentences['_gddid'] == article_id]
    return article[['sentid', 'words']].sort_values(by = 'sentid')

In [14]:
# Article 01 
order_article('550453fde1382326932d85f7')

,sentid,words
94865,1,"[Quaternary, Science, Revi..."
94866,2,"[Plant, macrofossil, data,..."
94867,3,"[Quantitative, palaeoclima..."
94868,4,"[The, chronological, frame..."
94870,5,"[Surface, conditions, in, ..."
...,...,...
95325,448,"[Vegetation, history, and,..."
95326,449,"[Quaternary, Science, Revi..."
95327,450,"[Zolitschka, ,, B., ,, &, ..."
95329,451,"[Sedimentology, ,, dating,..."


Skimmed info: 
 * `Article Name:` Palaeoclimate_chronology_and_vegetation_history_of_the_Weichselian_Lateglacial
 * `Link`: http://www.sciencedirect.com/science/article/pii/S0277379199000074  
 * `Site Name`:  Laghi di Monticchio
 * `Location`:  40° 56' 40" N, 15° 36' 48" E
 * `Age Span`: 
 * `Site Descriptions`: The record from Lago Grande di Monticchio now adds macrofossil evidence of the presence of relatively boreal tree taxa during the Lateglacial interstadial and provides a chronology based upon annually laminated sediments.

In [15]:
# Article 2
order_article('54b43266e138239d8684efed')

,sentid,words
0,1,"[Available, online, at, ww..."
1,2,"[The, Chihuahueños, Bog, r..."
2,3,"[An, Artemisia, steppe, ,,..."
3,4,"[C/N, ratios, ,, δ13C, and..."
4,5,"[Higher, percentages, of, ..."
...,...,...
605,572,"[Is, the, Valles, caldera,..."
144,573,"[Geology, 23, ,, 411, --, ..."
607,574,"[Wright, Jr., ,, H.E., ,, ..."
608,575,"[Present, and, past, veget..."


Skimmed info:  
* `Article:` Development of the Mixed Connifer Forest in Northern New Mexico and its relationship to Holocene environmental change
* `Link`: http://www.sciencedirect.com/science/article/pii/S0033589407001512
* `Site Name`:  Chihuahuenos Bog 
* `Location`:  ??? No Given Coordinates 
* `Age Span`:   
* `Site Descriptions`:  Chihuahueños Bog is located within the mixed conifer forest and has affinities to the flora of the southern Rocky Mountains(Foxx and Tierney, 1985; Allen, 2004). Although the area immediately surrounding the bog has been heavily logged in recent years, nearly every conifer species growing in montaneforests of the Jemez mountains is represented either along theshore of the bog or in the surrounding uplands.  

In [16]:
# Article 3
order_article('57c5b941cf58f1338eaddb5b')

,sentid,words
67708,1,"[A, record, of, sustained,..."
67709,2,"[Here, we, report, a, high..."
67710,3,"[Palynological, and, carbo..."
67712,4,"[Rapid, forest, clearance,..."
67713,5,"[Agricultural, intensiﬁcat..."
...,...,...
67845,123,"[Simon, ,, M.L., ,, and, P..."
67846,124,"[Smith, ,, B.D., ,, and, Y..."
67848,125,"[Sugita, ,, S., ,, 1993, ,..."
67849,126,"[Trubitt, ,, M.B.D., ,, 20..."


Skimmed info:  
* `Link`: http://dx.doi.org/10.1130/g35541.1  # No Full access to article
* `Site Name`:  Cahokia region, Illinois, USA
* `Location`:  N38°42'00" - N38°42'00", W90°04'60" - W90°04'60" - Not found on text (yet).
* `Age Span`:   
* `Site Descriptions`: Record from Horseshoe Lake, an oxbow lake in the central Mississippi River valley that is adjacent to the Cahokia site (Illinois, USA), the largest prehistoric settlement north of Mexico.   

In [17]:
# Article 4
order_article('58d29193cf58f14928755ba5')

,sentid,words
110659,1,"[Grana, ISSN, :, 0017-3134..."
110660,2,"[Peat-bog, Begbunar, -LRB-..."
110661,3,"[Peat-bog, Begbunar, -LRB-..."
110662,4,"[Submit, your, article, to..."
110665,5,"[Peat-bog, Begbunar, -LRB-..."
...,...,...
110740,80,"[Ann, .]"
110741,81,"[Sofia, Univ., .]"
110742,82,"[Fac, .]"
110743,83,"[Geol, .]"


In [18]:
# Try to see the exact location. Wont show last few words.
pd.DataFrame(order_article('58d29193cf58f14928755ba5').iloc[4])

,110665
sentid,5
words,"[Peat-bog, Begbunar, -LRB-..."


Skimmed info:  
* `Link:`http://www.tandfonline.com/doi/abs/10.1080/00173130902965157
* `Site Name`:  Begbunar
* `Location`:  42°09′ N, 22° 33′ E; 1750 ; in SQL file shows as: `42,°,09,cents,N,,,22,°,33,cents,E`
* `Age Span`:   
* `Site Descriptions`: Located in the central treeless zone of the Osogovo mountains, which are situated at the border between south-western Bulgaria and the north-eastern former Yugoslav Republic of Macedonia.

In [19]:
# Article 5
order_article('57928e07cf58f133d1c26609')

,sentid,words
39606,1,"[Timberline, fluctuations,..."
39607,2,"[By, tracking, climaticall..."
39608,3,"[Pollen, data, suggest, th..."
39609,4,"[The, inferred, climate, w..."
39611,5,"[Abies, -LRB-, fir, -RRB-,..."
...,...,...
40107,448,"[Weber, ,, W., A., ,, 1987..."
40108,449,"[Whitlock, ,, C., ,, 1993,..."
40110,450,"[Whitlock, ,, C., ,, and, ..."
40111,451,"[Wright, ,, H., E., ,, Jr...."


In [20]:
# Print the exact line where locations are.
pd.DataFrame(order_article('57928e07cf58f133d1c26609').iloc[179])

,39801
sentid,180
words,"[LOCATION, AND, AGE, OF, F..."


In [21]:
other_example = nlp_sentences[nlp_sentences['_gddid'] == '57928e07cf58f133d1c26609']
other_example = other_example[other_example['sentid'] == 180]
other_example[['words', 'liststring', 'dms_regex']]

,words,liststring,dms_regex
39801,"[LOCATION, AND, AGE, OF, F...","LOCATION,AND,AGE,OF,FOSSIL...","[38,°,49,′,50,′,′,106,°,24..."


Skimmed info:  
* `Link`: http://dx.doi.org/10.1130/0016-7606(1997)109<1306:tfalqp>2.3.co
* `Site Name`:  Cottonwood III, Cottonwood IV, Red Lady, Red Well, Splains
* `Location`:  38°49'50' 106°24'45', 38°49'50' 106°24'45', 38°52'50' 107°2'30', 38°52'50' 107°2'30',38°53'40' 107°3'15', 38°52'28' 107°05', 38°50' 107°4'30', 38°50' 107°4'30', 38°52' 107°2'30', 38°45' 106°50', 38°45' 106°50'
* `Age Span`:   17000 - 400
* `Site Descriptions`:   The lower timberline or lower forest border is the lower elevational limit of a continuous forest belt. In the Rocky Mountains it forms a sharp contrast with shrublands or grasslands in valley bottoms. Trees may extend below the lower timberline in favorable habitats, along streams or on rocky outcrops.  

In [22]:
# Find the article in the bibliography by gddid to retrieve Article Name

bibliography[bibliography['_gddid'] == '57928e07cf58f133d1c26609']

,_type,_id,publisher,title,journal.name.name,author,year,number,volumne,link,_gddid,type,pages,url
93,doi,10.1130/0016-7606(1997)109...,GSA,Timberline fluctuations an...,Geological Society of Amer...,"[{'name': 'Fall, Patricia ...",1997,10,NaN,[{'url': 'http://dx.doi.or...,57928e07cf58f133d1c26609,article,1306--1320,NaN


## Linking Neotoma DB to extract locations

In [58]:
neotoma = pd.read_csv("../Do_not_commit_data/data-1590729612420.csv")
neotoma = neotoma[['siteid', 'sitename', 'longitudeeast', 'latitudenorth', 'longitudewest', 'latitudesouth', 'sitedescription', 'doi']]
#neotoma.drop_duplicates(subset ="sitename", keep = False, inplace = True) 
neotoma

,siteid,sitename,longitudeeast,latitudenorth,longitudewest,latitudesouth,sitedescription,doi
0,10330,Lac du Sommet,-70.66468,47.71662,-70.66573,47.71382,The small shallow Lac du S...,10.1177/0959683611400199
1,10330,Lac du Sommet,-70.66468,47.71662,-70.66573,47.71382,The small shallow Lac du S...,10.1177/0959683611400199
2,1729,Myrtle Lake,-93.37853,47.98645,-93.39207,47.97876,Lake surrounded by peatlan...,10.1139/b68-190
3,1729,Myrtle Lake,-93.37853,47.98645,-93.39207,47.97876,Lake surrounded by peatlan...,10.1139/b68-190
4,269,Billy's Lake,-94.54948,46.27186,-94.55363,46.26930,Shallow depression on St. ...,10.2307/1939818
...,...,...,...,...,...,...,...,...
6884,26896,King & Kapp 1963 Site 19,-80.43829,47.49101,-80.43829,47.49101,"Site is near Makobe Lake, ...",10.1139/b63-022
6885,26896,King & Kapp 1963 Site 19,-80.43829,47.49101,-80.43829,47.49101,"Site is near Makobe Lake, ...",10.1139/b63-022
6886,525,Cupola Pond,-91.08914,36.79962,-91.09082,36.79864,Small sinkhole pond almost...,10.2307/2845468
6887,26913,Belaya Gora Horned Lark,147.16055,68.57887,147.16055,68.57887,The site comprises a serie...,10.1038/s42003-020-0806-7


In [63]:
df = nlp_sentences.merge(bibliography, on = '_gddid')

In [64]:
df[['liststring', 'journal.name.name', 'dms_regex']].head(100)

,liststring,journal.name.name,dms_regex
0,"Available,online,at,www.sciencedirect.com,Quaternary,Research,69,-LRB-,2008,-RRB-,263,--,275,www.elsevier.com/locate/yqres,Development,of,the,mixed,conifer,forest,in,northern,New,Mexico,and,its,relationship,to,Holocene,environmental,change,R.,Scott,Anderson,a,,,b,,,,,Renata,B.,Jass,b,,1,,,Jaime,L.,Toney,b,,2,,,Craig,D.,Allen,c,,,Luz,M.,Cisneros-Dozal,d,,,Marcey,Hess,d,,,Jeff,Heikoop,d,,,Julianna,Fessenden,d,a,Center,for,Environmental,Sciences,&,Education,,,Box,5694,,,Northern,Arizona,University,,,Flagstaff,,,AZ,86011,,,USA,b,Quaternary,Sciences,Program,&,Bilby,Research,Center,,,Box,6013,,,Northern,Arizona,University,,,Flagstaff,,,AZ,86011,,,USA,c,U.S.,Geological,Survey,,,Jemez,Mountains,Field,Station,,,Bandelier,National,Monument,,,HCR-1,,,Box,1,,,#,15,,,Los,Alamos,,,NM,87544,,,USA,d,Earth,&,Environmental,Sciences,Division,,,Hydrology,,,Geochemistry,&,Geology,Group,,,EES-6,,,MS-D462,,,Los,Alamos,National,Laboratory,,,Los,Alamos,,,NM,87545,,,USA,Received,28,April,2007,Available,online,29,January,2008,Abstract,Chihuahueños,Bog,-LRB-,2925,m,-RRB-,in,the,Jemez,Mountains,of,northern,New,Mexico,contains,one,of,the,few,records,of,late-glacial,and,postglacial,development,of,the,mixed,conifer,forest,in,southwestern,North,America,.",Quaternary Research,[]
1,"The,Chihuahueños,Bog,record,extends,to,over,15,000,cal,yr,BP,.",Quaternary Research,[]
2,"An,Artemisia,steppe,,,then,an,open,Picea,woodland,grew,around,a,small,pond,until,ca.,11,700,cal,yr,BP,when,Pinus,ponderosa,became,established,.",Quaternary Research,[]
3,"C/N,ratios,,,δ13C,and,δ15N,values,indicate,both,terrestrial,and,aquatic,organic,matter,was,incorporated,into,the,sediment,.",Quaternary Research,[]
4,"Higher,percentages,of,aquatic,algae,and,elevated,C/N,ratios,indicate,higher,lake,levels,at,the,opening,of,the,Holocene,,,but,a,wetland,developed,subsequently,as,climate,warmed,.",Quaternary Research,[]
5,"Anderson,,,R.S.,,,1989,.",Quaternary Research,[]
6,"From,ca.,8500,to,6400,cal,yr,BP,the,pond,desiccated,in,what,must,have,been,the,driest,period,of,the,Holocene,there,.",Quaternary Research,[]
7,"C/N,ratios,declined,to,their,lowest,Holocene,levels,,,indicating,intense,decomposition,in,the,sediment,.",Quaternary Research,[]
8,"Wetter,conditions,returned,after,6400,cal,yr,BP,,,with,conversion,of,the,site,to,a,sedge,bog,as,groundwater,levels,rose,.",Quaternary Research,[]
9,"Higher,charcoal,influx,rates,after,6400,cal,yr,BP,probably,result,from,greater,biomass,production,rates,.",Quaternary Research,[]


In [65]:
print(df.shape)

(111965, 24)


In [26]:
# Filter df to just frames that has REGEX longer than one
mask = (df['dms_regex'].str.len() > 0)
df = df.loc[mask]
df[['words','dms_regex']].head(100)

,words,dms_regex
3314,"[Lake, Hope, Simpson, -LRB...","[52,',27,',N,,,56,',26,',W]"
3318,"[The, lake, basin, itself,...","[52,',33,',N,,,56,',54,',W]"
3320,"[Moraine, Lake, -LRB-, 52,...","[52,',16,',N,,,58,',03,',W]"
3324,"[St., Paul, Lake, -LRB-, 5...","[52,',05,',N,,,57,',50,',W]"
3408,"[Diagram, of, pollen, perc...","[52,',16,',N,,,58,',03,',W]"
...,...,...
107854,"[Within, the, basin, ,, me...","[410,45,',N,410,40,',, 90,..."
109253,"[Bob, Black, Pond, and, Qu...","[2,2,,,9,0,0,]"
109436,"[C, III, --, I, ,, no., 5,...","[28,,,1,9,7,]"
109480,"[Nothofagus, antarctica, ,...","[710,26,',W,410,44,'S]"


In [57]:
# Neotoma Sitenames grouped by DOI

df4 = pd.DataFrame(neotoma.groupby('doi').apply(lambda x: list(x['sitename'])))
df4

,0
doi,
0.1139/e80-122,"[East Baltic Bog, Mermaid ..."
10.1002/(SICI)1099-1417(199607/08)11:4<327::AID-JQS257>3.0.CO;2-G,[Lundin Tower]
10.1002/(SICI)1099-1417(199701/02)12:1<15::AID-JQS283>3.0.CO;2-T,"[Spruce Pond, Spruce Pond,..."
10.1002/(SICI)1099-1417(199801/02)13:1<43::AID-JQS348>3.0.CO;2-G,"[Pantano de Genagra, Panta..."
10.1002/1099-1417(200010)15:7<687::AID-JQS551>3.0.CO;2-6,"[Laguna Loma Linda, Laguna..."
...,...
https://doi.org/10.1111/j.1502-3885.1991.tb00301.x,"[Mjällsjön, M. Lommesjön, ..."
https://doi.org/10.1111/j.1502-3885.2002.tb01051.x,"[Lake Medvedevskoye, Lake ..."
https://doi.org/10.1111/jbi.13628,"[Swallow Lagoon, Swallow L..."
